# 오토인코딩 언어모델

* 교제는 대부분 사전 훈련된 또는 미리 만들어진 모델을 사용하는 방법으로 특정한 모델과 그 훈련 방법에 관한 정보는 다루지 않는다.
* 오토인코딩(autoencoding, 자기부호화) 언어 모델을 처음부터 훈련하는 방법을 배운다. 
* 여기에는 모델의 일반적인 사전 훈련(pretraining)뿐만 아니라 특정 작업에 특화된 훈련도 포함한다. 
* 이번 장의 주제는 다음과 같음:
    * 오토인코딩 언어 모델 중 하나인 BERT(bidirectional encoder representations from transformers)
    * 임의의 언어를 위한 오토인코딩 언어 모델 학습
    * 모델을 커뮤니티와 공유하는 방법
    * 다른 여러 오토인코딩 모델 소개
    * 토큰화 알고리즘 다루기

## BERT: 오토인코딩 언어 모델의 하나

* BERT는 언어 모델링을 위해 인코더 트랜스포머 스택을 약간 수정하여 활용한 최초의 오토인코딩 언어 모델
* BERT 아키텍처는 원래 트랜스포머 구현에 기반한 다층 트랜스포머 인코더임. 트랜스포머 모델 자체는 원래 기계번역 작업을 위해 만들어졌지만, BERT의 주요 개선점은 이 아키텍처의 일부를 활용하여 더 나은 언어 모델링을 제공한다는 것
* 사전 훈련을 거치면 이 언어 모델은 학습된 언어를 종합적으로 이해하는 능력을 갖추게 됨

### BERT 언어 모델의 사전 학습 작업

* BERT가 사용하는 MLM(masked language modeling; 마스크 언어 모델링)을 명확히 이해하려면 MLM이라는 것을 알아야 함
* MLM은 일부 토큰이 마스킹된(가려진) 문장을 입력받아 그 토큰들을 채운 전체 문장을 출력으로 내는 작업
* 이런 모델링이 분류 같은 하위 작업의 성능을 향상하는 데 도움이 되는 이유는 무엇인가? 모델이 빈칸 추론 혹은 빈칸 채우기 문제(cloze test, 빈칸을 채워 언어 이해력을 평가하는 언어학적 테스트)를 풀 수 있다면, 그 모델은 언어에 대한 전반적인 이해를 갖추고 있다는 뜻
* 이미 마스크 언어 모델링을 통해 사전 학습이 이루어졌기 때문에 다른 작업에서도 더 나은 성능을 보일 수 있다.
* 예를 들어 다음과 같은 빈칸 채우기 문제를 생각해보면,

> 추석은 한국의 대표적인 명절 중 하나로, 음력 ____ 15일입니다.

* 빈칸에 들어갈 적절한 단어는 8월임. 마스크 언어 모델도 이와 마찬가지로, 문장에서 무작위로 선택된 마스킹된 토큰을 채우는 작업을 수행함
* BERT는 다음 문장 예측(next, sentence prediction, NSP) 작업에 대해서도 학습함. 이 사전 학습 작업을 통해 BERT는 마스킹된 토큰 예측 시 모든 토큰 사이의 관계를 학습함. 또한 이 사전 학습은 BERT 모델이 두 문장의 관계를 이해하는 데 도움이 됨. 이 사전 훈련에서 두 문장을 [SEP]라는 토큰으로 구분한 형태의 예시를 BERT에 제공함. 해당 데이터셋에는 둘째 문장이 첫 문장 뒤에 오는지 여부도 들어 있음
* 다음은 NSP 입력의 예:

> 이 문장은 빈칸 채우기가 필요하다. 비트코인은 다른 알트코인에 비해 가격이 너무 높다.

* 잘 훈련된 모델은 이 예시가 부정적이라고, 즉 두 문장이 서로 관련 없다고 예측할 것이다. 
* 이러한 두 가지 사전 훈력 작업을 통해 BERT는 언어 자체에 대한 이해를 얻게 됨. BERT의 토큰 임베딩은 각 토큰에 대한 문맥적 임베딩(contextual embedding; 또는 맥락적 임베딩)을 제공함. 
* 즉 각 토큰은 주변 토큰과 완전히 연관된 임베딩을 가짐. Word2Vec 같은 모델과 달리 BERT는 각 토큰 임베딩에 대해 더 풍부한 정보를 제공함. 또한 NSP 작업은 BERT가 [CLS] 토큰에 대해 더 나은 임베딩을 학습하도록 돕는다.
* [CLS] 토큰은 전체 입력에 관한 정보를 제공함. 이 토큰은 분류 작업에 사용되며, 사전 훈련 단계에서 전체 입력의 전반적인 임베딩을 학습하게 됨

### 자세히 살펴본 BERT 언어 모델

* 토크나이저는 여러 NLP 응용과 그 파이프라인에서 중요한 부분. BERT는 WordPiece라는 토큰화 알고리즘을 사용함
* WordPiece는 SentencePiece와 바이트 쌍 인코딩(byte pair encoding, BPE)과 함께 여러 트랜스포머 기반 아키텍처에서 가장 널리 알려진 세 가지 토크나이저 중 하나임
* 이들의 병합 전략, 단위 표현(바이트, 문자 또는 부분단어 단위), 토큰화 방식, 분할 모드의 유연성 면에서 차이가 있음. 알고리즘마다 나름 장단점이 있으므로 구체적인 작업의 요구사항에 맞게 선택해야 함
* BERT나 다른 트랜스포머 기반 아키텍처가 부분단어(subword) 토큰화 알고리즘을 사용하는 주된 이유는 그런 토크나이저가 미지의 토큰을 처리할 수 있기 때문

* BERT는 위치 인코딩을 이용해 토큰의 위치 정보를 모델에 제공. BERT나 유사 모델들은 밀집 신경망 층(dense neural network layer)같은 비순차적 연산을 사용함
* RNN이나 LSTM의 전통적인 모델은 자연스러운 위치를 고려함. BERT는 그런 추가 정보를 제공하는 데 위치 인코딩이 유용함

* BERT의 사전 훈련은 오토인코딩 모델과 유사하게 모델에 언어적 정보를 제공. 그러나 실제 응용에서 모델로 여러 문제(시퀀스 분류, 토큰 분류, 질의응답)를 풀 때는 출력의 서로 다른 부분이 사용됨
* 예를 들어 감성 분석이나 문장 분류 같은 시퀀스 분류 작업에서 원래 BERT 논문은 마지막 층의 CLS 임베딩을 사용해야 한다고 제안했음. 하지만 다른 연구에서 BERT에 다양한 기법을 적용해서 그런 분류 작업을 수행함
* 이를테면 모든 토큰의 평균 토큰 임베딩을 사용하거나 마지막 층에 LSTM을 적용하거나, 마지막 층 위에 CNN을 두는 식임. 시퀀스 분류를 위한 마지막 [CLS] 임베딩은 어떤 종류의 분류기에서도 사용할 수 있지만, 권장되는(그리고 가장 일반적인) 방법은 입력 크기가 최종 토큰 임베딩 크기와 같고 출력 크기가 클래스 수와 같은 소프트맥스 활성화 함수를 가진 밀집층을 사용하는 것
* 출력이 다중 레이블일 수 있고 문제 자체가 다중 레이블 분류인 문장에서 시그모이드 함수를 사용하는 것도 대안이 될 수 있음

![](bert_nsp.png)

![](bert_tasks.png)

* NER(named entity recognition) 작업은 문장 속에서 특정한 의미를 가진 개체(이름, 날짜, 장소 등)를 식별하고 분류하는 작업
* NER 작업의 경우 [CLS] 대신 각 토큰의 출력이 사용됨. 질의응답의 경우에는 질문과 답변이 구분 토큰 [SEP]로 연결되며, 답변은 마지막 층의 출력 중 '시작/끝' 구간(span)에 대한 주석(annotation)으로 주어짐. 이 경우 문단(paragraph)은 질문 대상에 관한 문맥(맥락)임
* BERT의 가장 중요한 능력은 문맥을 고려한 텍스트 표현임. 밀집 벡터 형태로 표현하는 트랜스포머 인코더 아키텍처 덕분으로 이 벡터들은 아주 간단한 분류기를 통해서 손쉽게 출력으로 변환됨

# BERT: As one of Autoencoding Language Models 

In [4]:
import pandas as pd

imdb_df = pd.read_csv("IMDB Dataset.csv")
reviews = imdb_df.review.to_string(index=None) 
with open("corpus.txt", "w", encoding="utf-8") as f: 
    f.writelines(reviews) 

In [7]:
from tokenizers import BertWordPieceTokenizer

bert_wordpiece_tokenizer = BertWordPieceTokenizer()
bert_wordpiece_tokenizer.train("corpus.txt") 
bert_wordpiece_tokenizer.get_vocab()

{'houseman': 15742,
 '##ckey': 11564,
 'under': 588,
 'bour': 9746,
 'knott': 10208,
 '##hel': 14353,
 'callahan': 12397,
 'reporters': 16023,
 'goldwyn': 12698,
 'mas': 2558,
 'maniac': 6273,
 'henriksen': 11097,
 'curios': 3964,
 'hig': 7134,
 'difficu': 12753,
 'wes': 2906,
 'paper': 4305,
 'schum': 9107,
 'viciously': 13571,
 'partial': 15400,
 'bachman': 16819,
 'axiom': 17379,
 'dix': 11986,
 'deranged': 10852,
 'musically': 12364,
 'masks': 12937,
 'sophie': 13110,
 '##gles': 9919,
 'believe': 657,
 'hooked': 4650,
 '##uttering': 17612,
 'manner': 7315,
 'oppos': 7602,
 'criticizing': 12819,
 '##lene': 14278,
 'lind': 3881,
 'putain': 15638,
 'expects': 10360,
 'herzog': 8135,
 'garg': 16011,
 'kicks': 8365,
 'sting': 14588,
 'aircraft': 16244,
 'reached': 7944,
 'suburban': 13398,
 'garage': 16012,
 'ds': 11375,
 'wanna': 6191,
 'mat': 936,
 'natali': 12841,
 'psycho': 5356,
 'play': 722,
 'willy': 16424,
 'ashamed': 6977,
 'blandings': 17159,
 'le': 427,
 'zoo': 11550,
 'pack'

`BertWordPieceTokenizer`에서 나오는 `"##"`는 **WordPiece 토크나이저가 단어를 부분(subword) 단위로 나눌 때 사용하는 접두사**입니다.  
이건 BERT가 **모르는 단어**나 **희귀한 단어**를 처리하기 위해 사용하는 방식입니다.

---

#### 예시

예를 들어 다음과 같은 문장이 있다고 해봅시다:

> `unbelievable`

#### 토크나이저 처리 결과:
```
["un", "##bel", "##iev", "##able"]
```

- `"un"`: 어근으로 잘 알려진 토큰 → 그대로 사용
- `"##bel"`: 단독으로는 보기 힘든 토큰 → "bel" 앞에 ## 붙여서 연결 의미 표현
- `"##iev"`: 역시 부분 토큰
- `"##able"`: 어미 부분

이렇게 나누면 `["un", "##bel", "##iev", "##able"]`이 순서대로 이어져 **"unbelievable"이라는 원래 단어를 복원할 수 있음**.

#### 왜 이런 걸 쓰는가?

- 단어 전체를 사전에 넣으면 vocabulary가 너무 커짐 → **효율성 문제**
- 새로운 단어(신조어, 오타 등)도 **부분 단위로 인식 가능** → **일반화 성능 향상**

#### 주의할 점

- `"##"`는 해당 서브토큰이 **단어의 중간이나 끝부분**이라는 것을 의미함
- `"##"`가 없는 토큰은 **단어의 시작** 또는 **독립된 단어**라는 뜻임

#### 실제 활용

```python
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize("unbelievable happiness")
print(tokens)
# 출력: ['un', '##bel', '##iev', '##able', 'happiness']
```

In [8]:
bert_wordpiece_tokenizer.save_model("tokenizer")

['tokenizer\\vocab.txt']

In [11]:
tokenizer = BertWordPieceTokenizer.from_file("tokenizer/vocab.txt")
tokenized_sentence = tokenizer.encode("Oh it works just fine")
print(tokenized_sentence.tokens)
tokenized_sentence = tokenizer.encode("ohoh i thougt it might be workingg well")
print(tokenized_sentence.tokens)

['[CLS]', 'oh', 'it', 'works', 'just', 'fine', '[SEP]']
['[CLS]', 'oh', '##o', '##h', 'i', 'thoug', '##t', 'it', 'might', 'be', 'working', '##g', 'well', '[SEP]']


transformers 라이브러리의 `BertTokenizerFast`를 사용함. 이것을 사용하는 것은 허깅페이스 문서에서 권장하기 때문

In [1]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("tokenizer")

In [2]:
from transformers import LineByLineTextDataset

# block_size는 모델이 한 번에 처리할 수 있는 토큰 수를 결정함. 이 값은 모델의 크기에 따라 다름
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path="corpus.txt", block_size=128)

c:\Users\unerue\Documents\tutorial_transformers\.venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [3]:
dataset.examples[:3]

[{'input_ids': tensor([   2,  195,  146,  136,  587, 1735,  264, 3421,  191,   18,   18,   18,
             3])},
 {'input_ids': tensor([   2,   43,  774,  530, 1097,   18,   32,  196,   19,   34,   32,  196,
            19,   34,  136,   18,   18,   18,    3])},
 {'input_ids': tensor([   2,   51,  480,  138,  176,   43,  774,  662,  159, 2622, 1854,   18,
            18,   18,    3])}]

In [4]:
from transformers import DataCollatorForLanguageModeling

# mlm은 마스킹 언어 모델링(masked language modeling)을 의미함. 이 작업은 모델이 토큰을 예측하도록 학습하는 것을 의미함
# mlm_probability는 마스킹된 토큰의 비율을 결정함. 이 값은 모델의 크기에 따라 다름
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [5]:
import torch

torch.cuda.is_available()

True

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="bert",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=128,
)

In [8]:
from transformers import BertConfig, BertForMaskedLM

print(BertConfig())
bert = BertForMaskedLM(BertConfig())

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [9]:
from transformers import Trainer

trainer = Trainer(
    model=bert,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
) 

In [10]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=391, training_loss=5.388594449328645, metrics={'train_runtime': 183.8619, 'train_samples_per_second': 272.063, 'train_steps_per_second': 2.127, 'total_flos': 630581155416000.0, 'train_loss': 5.388594449328645, 'epoch': 1.0})

In [11]:
trainer.save_model("bert_imdb")

In [12]:
from transformers import BertConfig

BertConfig()

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [13]:
tiny_bert_config = BertConfig(
    max_position_embeddings=512,
    hidden_size=128,
    num_attention_heads=2,
    num_hidden_layers=2,
    intermediate_size=512
) 
tiny_bert_config 

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [14]:
tiny_bert = BertForMaskedLM(tiny_bert_config) 
trainer = Trainer(
    model=tiny_bert,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=391, training_loss=8.847049332640665, metrics={'train_runtime': 15.4527, 'train_samples_per_second': 3237.098, 'train_steps_per_second': 25.303, 'total_flos': 3268431345600.0, 'train_loss': 8.847049332640665, 'epoch': 1.0})

In [16]:
from transformers import BertModel, BertTokenizerFast

bert = BertModel.from_pretrained("bert-base-uncased") 
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [17]:
bert.encoder.layer

ModuleList(
  (0-11): 12 x BertLayer(
    (attention): BertAttention(
      (self): BertSdpaSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [22]:
tokenized_text = tokenizer.batch_encode_plus(
    ["hello how is it going with you", "lets test it"],
    return_tensors="pt",
    max_length=256,
    truncation=True,
    pad_to_max_length=True
)
bert(**tokenized_text)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 1.0047e-01,  6.7703e-02, -8.3359e-02,  ..., -4.9330e-01,
           1.1654e-01,  2.2665e-01],
         [ 3.2362e-01,  3.7072e-01,  6.1469e-01,  ..., -6.2727e-01,
           3.7908e-01,  7.0530e-02],
         [ 1.9953e-01, -8.7551e-01, -6.4786e-02,  ..., -1.2808e-02,
           3.0765e-01, -2.0733e-02],
         ...,
         [-6.5330e-02,  1.1905e-01,  5.7685e-01,  ..., -2.9546e-01,
           2.4974e-02,  1.1396e-01],
         [-2.6471e-01, -7.8639e-02,  5.4728e-01,  ..., -1.3752e-01,
          -5.9468e-02, -5.1793e-02],
         [-2.4496e-01, -1.1480e-01,  5.9217e-01,  ..., -1.5688e-01,
          -3.3976e-02, -8.4614e-02]],

        [[ 2.9456e-02,  2.3087e-01,  2.9265e-01,  ..., -1.3042e-01,
           1.8966e-01,  4.6843e-01],
         [ 1.7052e+00,  6.9136e-01,  7.3151e-01,  ...,  2.8930e-01,
           5.3676e-01, -1.5455e-01],
         [ 1.0460e-01,  9.6368e-02,  6.9967e-02,  ..., -4.1592e-01,
          -1.

In [49]:
import torch
import torch.nn as nn
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased'):
        super().__init__()
        self.max_length = 256
        
        # BERT 모델 로드
        self.bert = BertModel.from_pretrained(bert_model_name)
        
        # 분류를 위한 Dense 레이어(softmax 포함)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input_ids, attention_mask, labels=None):
        # BERT의 출력에서 [CLS] 토큰의 임베딩만 사용
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # [CLS] 토큰 선택
        
        logits = self.classifier(cls_embedding)
        predictions = self.softmax(logits)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

        return {'loss': loss, 'logits': logits} if loss is not None else {'logits': logits}

# 모델 초기화
model = BertClassifier()

In [47]:
tokenized = tokenizer.batch_encode_plus(
    ["hello how is it going with you", "hello how is it going with you"],
    return_tensors="pt",
    max_length=256,
    truncation=True,
    pad_to_max_length=True
) 

In [50]:
model(input_ids=tokenized["input_ids"], attention_mask=tokenized["attention_mask"])

{'logits': tensor([[ 0.2501, -0.0917],
         [ 0.2501, -0.0917]], grad_fn=<AddmmBackward0>)}

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset

In [74]:
# IMDB 데이터셋 로드 (캐글 버전)
df = pd.read_csv('IMDB Dataset.csv')  # sentiment, review 컬럼
df['label'] = (df['sentiment'] == 'positive').astype(int)  # positive: 1, negative: 0

# 학습/검증 분할
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 2. 데이터셋 변환
def convert_to_dataset(df):
    dataset_dict = {
        'text': df['review'].tolist(),
        'labels': df['label'].tolist()  # 반드시 'labels'로 키 이름 지정
    }
    return Dataset.from_dict(dataset_dict)

train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)

# 3. 토크나이저 초기화
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# 4. 데이터 전처리 함수
def preprocess_function(examples):
    # 텍스트 토크나이징
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=256,
    )
    
    # 레이블 추가
    tokenized['labels'] = examples['labels']

    return tokenized

# 데이터셋 전처리
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)
tokenized_valid = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [76]:
from transformers import PreTrainedModel

class IMDBBertClassifier(PreTrainedModel):
    def __init__(self, config: BertConfig):
        super().__init__(config)
        self.bert = BertModel.from_pretrained('bert-base-uncased', num_labels=2)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(config.hidden_size, 2)
        
    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))
        
        if loss is not None:
            return {
                'loss': loss,
                'logits': logits
            }
        else:
            return {
                'logits': logits
            }

In [77]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='binary')
    }

In [78]:
training_args = TrainingArguments(
    output_dir="./imdb_results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./imdb_logs",
    logging_steps=100,
    learning_rate=2e-5,
    # 평가 및 저장 설정
    eval_steps=500,  # 500스텝마다 평가
    save_steps=500,  # 500스텝마다 저장
    save_total_limit=2,  # 최근 2개의 체크포인트만 저장
    # GPU 설정
    no_cuda=False,
    fp16=True,  # 16비트 학습으로 메모리 절약
    # 기타 설정
    remove_unused_columns=True,
    dataloader_num_workers=4,  # 데이터 로딩 병렬화
    gradient_accumulation_steps=2,  # 그래디언트 누적으로 큰 배치 효과
    # 평가 활성화
    do_eval=True,  # 평가 수행
    do_train=True,  # 학습 수행
)

In [72]:
from transformers import TrainerCallback


config = BertConfig.from_pretrained('bert-base-uncased')
model = IMDBBertClassifier(config)


class IMDBTrainingCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"\n에포크 {state.epoch} 시작!")
        
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        print("\n평가 결과:")
        print(f"스텝: {state.global_step}")
        print(f"검증 정확도: {metrics.get('eval_accuracy', 0):.4f}")
        print(f"검증 F1 점수: {metrics.get('eval_f1', 0):.4f}")
        print(f"검증 손실: {metrics.get('eval_loss', 0):.4f}")

In [79]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
    # callbacks=[IMDBTrainingCallback()]
)
trainer.train()

Step,Training Loss
100,0.662600
200,0.394500
300,0.315000
400,0.285300
500,0.260100
600,0.263000
700,0.253900
800,0.238200
900,0.217300
1000,0.214000


TrainOutput(global_step=3750, training_loss=0.17416315434773763, metrics={'train_runtime': 1402.8183, 'train_samples_per_second': 85.542, 'train_steps_per_second': 2.673, 'total_flos': 1.57866633216e+16, 'train_loss': 0.17416315434773763, 'epoch': 3.0})